In [3]:
from openfermion.chem import MolecularData
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.circuits import uccsd_singlet_paramsize
from qiskit import Aer, execute

# Drug design

In [6]:
# oxalic acid geometry
#       O
#       |
#       C - C - O - H
#       |   
#       O - H
# Bond lengths in Angstroms
# C-O bond length is 1.43 Angstroms 
# C-C bond length is 1.45 Angstroms
# O-H bond length is 0.96 Angstroms

# set the Molecule for Oxalic Acid
geometry = [('C', (0., 0., 0.)),
            ('O', (0., 0., 1.43)),
            ('O', (0., 0., -1.43)),
            ('C', (1.45, 0., 0.)),
            ('O', (2.48, 0., 0.)),
            ('H', (3.44, 0., 0.)),
            ('H', (0., 0.96, -1.43))]

basis = 'sto-3g'
multiplicity = 1
charge = 0

# set the Molecule for Oxalic Acid
molecule = MolecularData(geometry, basis, multiplicity, charge)
molecule.load()
# compute the molecular Hamiltonian
fermion_hamiltonian = get_fermion_operator(molecule.get_molecular_hamiltonian())
qubit_hamiltonian = jordan_wigner(fermion_hamiltonian)

# Use the UCCSD ansatz
n_elecs = molecule.n_electrons
n_orbitals = molecule.n_orbitals
initial_state = uccsd_singlet_paramsize(n_orbitals, n_elecs)

# Use the Qiskit Aer simulator
backend = Aer.get_backend('statevector_simulator')
result = execute(initial_state, backend).result()
initial_amplitudes = result.get_statevector()

result = quantum_molecular_dynamics(qubit_hamiltonian, initial_amplitudes)

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'C:\Users\mgabi\AppData\Local\Programs\Python\Python39\Lib\site-packages\openfermion\testing\data/H2-C2-O3_sto-3g_singlet.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)